In [6]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
import watson_developer_cloud.natural_language_understanding.features.v1 \
  as Features

def watson(claim):
    natural_language_understanding = NaturalLanguageUnderstandingV1(
      username="09b56387-57ee-4390-9365-a07a37706fb4",
      password="ISoTe5EueZJp",
      version="2017-02-27")

    response = natural_language_understanding.analyze(
      text = claim,
      features=[
        Features.Keywords(
          emotion=False,
          sentiment=True,
            limit=15
        )
      ]
    )
    keywords = []
    for keyword in response['keywords']:
        if keyword['relevance'] > 0.80 and len(keywords) < 8:
            keywords.append(keyword['text'].encode('utf-8'))
    print response
    return keywords

url = 'http://abcnews.go.com/Politics/wireStory/trump-supporters-critics-juggalos-descend-washington-49891059?cid=clicksource_4380645_1_hero_headlines_headlines_hed'
claim = 'Big business got Brazil hooked on junk food'
print watson(claim)

{u'usage': {u'text_characters': 43, u'features': 1, u'text_units': 1}, u'keywords': [{u'relevance': 0.90279, u'text': u'junk food', u'sentiment': {u'score': -0.779874}}, {u'relevance': 0.766517, u'text': u'Big business', u'sentiment': {u'score': -0.779874}}, {u'relevance': 0.52341, u'text': u'Brazil', u'sentiment': {u'score': -0.779874}}], u'language': u'en'}
['junk food']


In [4]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
import watson_developer_cloud.natural_language_understanding.features.v1 \
  as Features

def watson(claim):
    natural_language_understanding = NaturalLanguageUnderstandingV1(
      username="09b56387-57ee-4390-9365-a07a37706fb4",
      password="ISoTe5EueZJp",
      version="2017-02-27")

    response = natural_language_understanding.analyze(
      text = claim,
      features=[
        Features.Concepts(
          # Concepts options
          limit=3
        )
      ]
    )
    print response

url = 'http://abcnews.go.com/Politics/wireStory/trump-supporters-critics-juggalos-descend-washington-49891059?cid=clicksource_4380645_1_hero_headlines_headlines_hed'
claim = 'Big business got Brazil hooked on junk food'
print watson(claim)
    

{u'usage': {u'text_characters': 43, u'features': 1, u'text_units': 1}, u'language': u'en', u'concepts': [{u'relevance': 0.885924, u'text': u'Nutrition', u'dbpedia_resource': u'http://dbpedia.org/resource/Nutrition'}, {u'relevance': 0.8188, u'text': u'Junk food', u'dbpedia_resource': u'http://dbpedia.org/resource/Junk_food'}]}
None


In [9]:
from eventregistry import *
from threading import Thread, Lock
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from py_ms_cognitive import PyMsCognitiveWebSearch
import watson_developer_cloud.natural_language_understanding.features.v1 as Features
import nltk
import pandas as pd
import json

# Print a list of recently added articles mentioning entered words
api_key = 'eda39267-9017-481a-860d-0b565c6d8bf3'
er = EventRegistry(apiKey = api_key)

global_df = pd.DataFrame()
mutex = Lock()
claim = ''

# Given keywords, this funciton appends the article metadata to the global pandas dataframe
def get_articles(keywords):
    global global_df
    global claim
    q = QueryArticlesIter(keywords=QueryItems.AND(keywords))
    q.setRequestedResult(RequestArticlesInfo(count= 199, sortBy="sourceImportance"))

    x = 0

    local_df = pd.DataFrame()

    res = er.execQuery(q)
    for article in res['articles']['results']:
        if x is 0:
            claim = article['title'].encode('utf-8'),
        data = {
            'source': article['source']['title'].encode('utf-8'),
            'url' : article['url'].encode('utf-8'),
            'text' : article['body'].encode('utf-8')
        }

        local_df = pd.concat([local_df, pd.DataFrame(data,index=[x])])
        x += 1

    mutex.acquire()
    try:
        global_df = pd.concat([global_df,local_df])
    finally:
        mutex.release()

# Given a url, this function returns up to 15 keywords
def watson(user_url):
    natural_language_understanding = NaturalLanguageUnderstandingV1(
      username="09b56387-57ee-4390-9365-a07a37706fb4",
      password="ISoTe5EueZJp",
      version="2017-02-27")

    response = natural_language_understanding.analyze(
      url=user_url,
      features=[
        Features.Keywords(
          emotion=False,
          sentiment=False,
            limit=15
        )
      ]
    )
    keywords = []
    for keyword in response['keywords']:
        if keyword['relevance'] > 0.80 and len(keywords) < 8:
            keywords.append(keyword['text'].encode('utf-8'))
    return keywords

# Worker thread class override
class myThread(threading.Thread):
    def __init__(self, query):
        threading.Thread.__init__(self)
        self.query = query

    def run(self):
        get_articles(self.query)

# given claim, azure returns related urls using bing searches
def azure_search(claim):
    search_term = claim
    search_service = PyMsCognitiveWebSearch('75d1a40af4bf4ba4bdf561ae25b5db5c', claim)
    first_three_result = search_service.search(limit=3, format='json') #1-50

    urls = []
   # To get individual result json:
    for i in first_three_result:
        urls.append(i.url.encode('utf-8'))
    return urls

# given a list of urls, this function returns all related keywords for the urls
def azure_claim(urls):
    keywords = []
    for url in urls:
        keywords.append(watson(url))
    print keywords
    return keywords

# given keywords, query event registry and append to global dataframe
def watson_azure_scrape(keywords):
    global global_df

    index = 0
    threads = []

    for query in keywords:
        threads.append(myThread(query))
        threads[index].start()
        index += 1
    for thread in threads:
        thread.join()
    global_df = global_df.reset_index(drop=True)
    return global_df
#     global_df.to_csv('watson_articles.csv')
#     global_df['uid'] = range(len(global_df.index))
#     return global_df.to_dict(orient='records')

# Call this function with a claim to query event registry
def run_azure(claim):
    claim_tokens = nltk.word_tokenize(claim)
    if len(claim_tokens) is 3:
        # Go straight to event registry with claim
        watson_azure_scrape(claim)
    else:
        watson_azure_scrape(azure_claim(azure_search(claim)))

# Call this function with a url to query event registry
def watson_scrape(url):
    global global_df
    keywords = watson(url)

    index = 0
    threads = []

    for query in keywords:
        threads.append(myThread(query))
        threads[index].start()
        index += 1
    for thread in threads:
        thread.join()
    global_df = global_df.reset_index(drop=True)
    return global_df
    # global_df.to_csv('watson_articles.csv')
#     global_df['uid'] = range(len(global_df.index))
#     return global_df.to_dict(orient='records')

url = 'http://abcnews.go.com/Politics/wireStory/trump-supporters-critics-juggalos-descend-washington-49891059?cid=clicksource_4380645_1_hero_headlines_headlines_hed'
# print watson_scrape(url)

claim = 'IBM watson is not very intelligent'
# print run_azure(claim)

print azure_claim(azure_search(claim))

using user provided API key for making requests
Event Registry host: http://eventregistry.org
[['requests'], ['gold stocks', 'Watson Analytics'], ['Bob Dylan']]
[['requests'], ['gold stocks', 'Watson Analytics'], ['Bob Dylan']]
